In [3]:
#Important Libraries

from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup

In [4]:
def getData(url):
    Booklist = list()
    Authorlist = list()
    BookDescriptions = list()
    BookGenres = list()
    count = 0

    # read the website url 
    try:
        html = urlopen(url)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print("The server could not be found!")
    else:
        print(html)

    # initialise the object for webpage
    bs = BeautifulSoup(html.read(),"html.parser")

    #get the book titles
    booktitles = bs.findAll('a', class_='bookTitle')
    print(f"We have found {len(booktitles)} Documents")
    
    for booktitle in booktitles:
        # get the url of each book
        url = booktitle.get("href")
        if url !="":
            booklisturl = 'https://www.goodreads.com{}'.format(url)

            try:
                # get the the url inside booktitle
                booklistpage =BeautifulSoup(urlopen(booklisturl).read(),"html.parser")

                #extract the book title
                title = booklistpage.find('h1', class_='Text Text__title1').get_text()
                Booklist.append(title)

                #extract the book author
                author = booklistpage.find('h3', class_='Text Text__title3 Text__regular').get_text()
                Authorlist.append(author)

                #extract the book description
                description = booklistpage.find('div', class_='DetailsLayoutRightParagraph__widthConstrained').get_text()
                BookDescriptions.append(description)

                #extract the book genre
                genres_tag = booklistpage.findAll('span', class_='BookPageMetadataSection__genreButton')
                genres = []
                for genre in genres_tag:
                    genres.append(genre.get_text())
                BookGenres.append(genres)

                # track the number of documents scrapped for given url
                count+=1
            except:
                pass

    return Booklist, Authorlist, BookDescriptions, BookGenres, count

In [5]:
urls =[f"https://www.goodreads.com/list/show/101.Best_Book_Cover_Art?page={i}" for i in range(1,2)] #get all the urls for scrapping the data

Booklist = list() # list for book titles
Authorlist = list() # list for book author
BookDescriptions = list() # list for book description
BookGenres = list() # list for book genre
total = 0 # counter for tracking the number of documents being scrapped

for data in urls:
    booklist, authorlist, book_descriptions, book_genres, count = getData(data) #get all the data from url

    # add the data in corresponding list
    Booklist.extend(booklist) 
    Authorlist.extend(authorlist)
    BookDescriptions.extend(book_descriptions)
    BookGenres.extend(book_genres)

    #count the document number
    total +=count

    #Scrape atleast 5000 documents and end the process
    if total>=200:
        break
    print("Total Scrapped Documents:",total)

We have found 100 Documents
Total Scrapped Documents: 100


In [6]:
import pandas as pd
df =pd.DataFrame({"Book Title":Booklist,"Author":Authorlist,"Description":BookDescriptions,"Genre":BookGenres})
df.to_csv("BookData.csv")

In [7]:
df.shape #get dimessions of the data scrapped

(100, 4)